In [11]:
from pynq import Overlay
from pynq.lib import AxiGPIO
import numpy as np
import time
from tqdm.notebook import tqdm

ol = Overlay("top.bit")

In [18]:
io_ctrl  = ol.ip_dict['axi_gpio_ctrl']
clk  = AxiGPIO(io_ctrl).channel1
rstn = AxiGPIO(io_ctrl).channel2

io_odata_opc  = ol.ip_dict['axi_gpio_dout']
io_rdata_out  = AxiGPIO(io_odata_opc).channel1
io_raddr_out  = AxiGPIO(io_odata_opc).channel2

io_idata    = ol.ip_dict['axi_gpio_din1']
io_waddr_in = AxiGPIO(io_idata).channel1
io_wdata_in = AxiGPIO(io_idata).channel2

io_ipc_state    = ol.ip_dict['axi_gpio_din2']
io_raddr_in     = AxiGPIO(io_ipc_state).channel1
io_wen_state_in = AxiGPIO(io_ipc_state).channel2

In [19]:
def clk_event():
    clk.write(0, 0b1)
    #time.sleep(0.1)
    clk.write(1, 0b1)
    #time.sleep(0.1)
    clk.write(0, 0b1)
    
def mem_read(mem, addr):
    addr_max = len(mem)
    ret = np.uint32(0)
    for i in range(4):
        ret += (mem[addr+i] << (i * 8)) if addr + i < addr_max else 0
    return ret.item()

def mem_write(mem, addr, data):
    addr_max = len(mem)
    for i in range(4):
        if (addr+i) < addr_max:
            mem[addr+i] = (data >> (i*8)) & 0xff

In [21]:
raddr = 0
mem = np.fromfile("./lb.bin", np.uint8)

rstn.write(0, 0x01)
clk_event()
rstn.write(1, 0x01)

for i in tqdm(range(1000)):
    rdata = mem_read(mem, raddr)
    io_raddr_out.write(raddr, 0xffffffff)
    io_rdata_out.write(rdata, 0xffffffff)
    #time.sleep(0.1)
    waddr = io_waddr_in.read()
    wdata = io_wdata_in.read()
    wen   = io_wen_state_in.read() & 0x1
    raddr = io_raddr_in.read()
    if (wen == 1):
        #print("{:08x}: {:08x}".format(w_addr, w_data))
        mem_write(mem, waddr, wdata)
    clk_event()
    
print("Value at mem[0x1000](0x11111111 for success): {:#10x}".format(mem_read(mem, 0x1000)))

  0%|          | 0/1000 [00:00<?, ?it/s]

Value at mem[0x1000](0x11111111 for success): 0x11111111
